In [5]:
# Load necessary library and module
import warnings
warnings.filterwarnings('ignore')

import sys
import pandas as pd
import seaborn as sns
import numpy as np
import re
import string
import unicodedata
import nltk

nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from textblob import TextBlob, Word


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Read and show sample data
mlbb = pd.read_csv('dataset/baruambil-20062022-MLBB.csv', encoding='ISO-8859-1')
hdi = pd.read_csv('dataset/baruambil-20062022-HDI.csv', encoding='ISO-8859-1')
ffm = pd.read_csv('dataset/baruambil-20062022-FreeFireMax.csv', encoding='ISO-8859-1')

In [7]:
# Apply text pre-processing to DataFrame
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
snowball = nltk.stem.SnowballStemmer('english')
porter_stemmer = PorterStemmer()
nltk.download('words')
from textblob import TextBlob, Word

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [8]:
def text_preprocessing(text):
  text = text.lower()                                   # Lowercase all sentences
      #   text = text.strip()                                   # Remove whitespace
  text = re.sub('[-+]?[0-9]+', ' ', text)               # Remove numbers 
  text = re.sub(r'https?://\S+|www\.\S+', ' ', text)    # Remove URLs
      #   text = re.sub(r"pic.twitter.com\S+", ' ', text)       # Remove custom URLs for twitter
      #   text = re.sub(r'\@([\w]+)',' ', text)                 # Remove Mention @
      #   text = re.sub(r'\#([\w]+)',' ', text)                 # Remove #TAGAR
  text = re.sub('\S*@\S*\s?', ' ', text)                # Remove email
  text = re.sub(r'[^\w\s]', ' ', text)                  # Remove punctuation
      #   text = re.sub(r'\b\w{1,3}\b','',text)                 #Remove n-chars,Remove less than 3 chars, minimum 4 character allowed "\b[a-zA-Z0-9]{3}\b"
  text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text)  # Tahap-5: simbol
      #   text = re.sub(r'[0-9]+','', text)                     # Tahap-6: angka
  text = re.sub(r'([a-zA-Z])\1\1','\\1', text)          # Tahap-7: koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
  text = re.sub(' +',' ', text)                         #remove multiple whitespace
  text = re.sub(r'^[ ]|[ ]$','', text)                  # Tahap-9: spasi di awal dan akhir kalimat

  # text = re.sub('\b[a-zA-Z0-9]{3}\b','',text)
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') # Remove non-ascii character
  word_tokens = word_tokenize(text) # Word tokenize
  

  # Define Indonesian stopwords removal
  stop_words = stopwords.words('english')  # NLTK Indonesian stopwords
  clean_words = [word for word in word_tokens if word not in stop_words] # stopwords removal
  clean_words = ' '.join(clean_words)

  words = set(nltk.corpus.words.words())
  word_cleaner = word_tokenize(clean_words)
  cleaner_words = [w for w in word_cleaner if w.lower() in words or not w.isalpha()] #remove non english
  cleaner = ' '.join(cleaner_words)

  #stemming with snowball update 07072022 stemmer terakhir
#   stem_token=word_tokenize(cleaner)
#   stem_words = [snowball.stem(w) for w in stem_token]
#   stem_clean= ' '.join(stem_words)

  #update13072022 pake lemmatization TextBlob
  tblob=TextBlob(clean_words)
  lemma= [Word(word).lemmatize("v") for word in tblob.words]
  lemma_clean= ' '.join(lemma)

  #   cleaner = " ".join(w for w in nltk.wordpunct_tokenize(clean_words) \
        #   if w.lower() in words or not w.isalpha())

  return lemma_clean

In [9]:
%%time
mlbb['clean_content'] = mlbb['content'].progress_apply(lambda x: text_preprocessing(x))
ffm['clean_content'] = ffm['content'].progress_apply(lambda x: text_preprocessing(x))
hdi['clean_content'] = hdi['content'].progress_apply(lambda x: text_preprocessing(x))

  0%|          | 0/1200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
mlbb.to_csv( "hasil/1_mlbb_clean_content_lemma_13072022_bab4.csv", index=False, encoding='utf-8-sig')
ffm.to_csv( "hasil/1_ffm_clean_content_lemma_13072022_bab4.csv", index=False, encoding='utf-8-sig')
hdi.to_csv( "hasil/1_hdi_clean_content_lemma_13072022_bab4.csv", index=False, encoding='utf-8-sig')